In [1]:
from snorkel.labeling import LabelingFunction
from snorkel.labeling import labeling_function
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.analysis import get_label_buckets
from snorkel.labeling.model import MajorityLabelVoter
from snorkel.labeling import filter_unlabeled_dataframe
import re

In [2]:
#import csv file and load train/test/split of dataset
from utils import load_dataset
df_train, df_test = load_dataset()
# df_train = df_train.fillna("")
Y_test = df_test.label.values

In [6]:
#get lfs
from snorkel_get_lfs import lfs
applier = PandasLFApplier(lfs=lfs)

# define train and test sets
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

100%|██████████| 2/2 [00:00<00:00, 401.14it/s]


In [17]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
keyword_my,0,[1],0.250000,0.166667,0.0
keyword_subscribe,1,[1],0.083333,0.083333,0.0
keyword_http,2,[1],0.083333,0.000000,0.0
keyword_please,3,[1],0.166667,0.083333,0.0
keyword_song,4,[],0.000000,0.000000,0.0
regex_check_out,5,[],0.000000,0.000000,0.0


In [35]:
# buckets = get_label_buckets(L_train[:, 0], L_train[:, 1])
# df_train.iloc[buckets[(ABSTAIN, SPAM)]].sample(10, random_state=1)

In [19]:
majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)

In [21]:
label_model = LabelModel(cardinality=3, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)


In [28]:
L_train

array([[ 1, -1, -1,  1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       [-1, -1,  1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       [-1, -1, -1,  1, -1, -1],
       [ 1,  1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       [ 1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1]])

In [29]:
L_test

array([[-1, -1, -1, -1, -1, -1],
       [-1, -1, -1,  1, -1, -1]])

In [30]:
Y_test

array([-1., -1.])

In [36]:
majority_acc = majority_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]

print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
     "accuracy"
 ]

print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")


LabelModel()


C:\Users\ARalevski\miniconda3\envs\snorkel\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\ARalevski\miniconda3\envs\snorkel\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\ARalevski\miniconda3\envs\snorkel\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\ARalevski\miniconda3\envs\snorkel\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [23]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
keyword_my,0,[1],0.250000,0.166667,0.0
keyword_subscribe,1,[1],0.083333,0.083333,0.0
keyword_http,2,[1],0.083333,0.000000,0.0
keyword_please,3,[1],0.166667,0.083333,0.0
keyword_song,4,[],0.000000,0.000000,0.0
regex_check_out,5,[],0.000000,0.000000,0.0


In [24]:
df_train_filtered, preds_train_filtered = filter_unlabeled_dataframe(
    X=df_train, y=preds_train, L=L_train)
    

In [25]:
df_train["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")

In [39]:
label_model.save("snorkel_2.pkl")

In [40]:
df_train.to_csv("results_keyword_test.csv")